# Import necessary packages

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import mean_absolute_error, mean_squared_error

from timeit import default_timer as timer

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
sales_train_val = sales_train_val[(sales_train_val['state_id'] == 'CA') & (sales_train_val['cat_id'] == 'HOBBIES')]

# Import original data and preprocessed data

In [3]:
sales_train_val = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sales_train_validation.csv')

In [4]:
sales_train_val = sales_train_val[(sales_train_val['state_id'] == 'CA') & (sales_train_val['cat_id'] == 'HOUSEHOLD')]

In [5]:
INPUT_DIR_2 = '/content/drive/MyDrive/Colab Notebooks/List of Product ID according to 4 demand patterns/California/Household/2-year'
list_intermittent = pd.read_csv(f'{INPUT_DIR_2}/Intermittent_ID_2_Year_Data.csv')
list_lumpy = pd.read_csv(f'{INPUT_DIR_2}/Lumpy_ID_2_Year_Data.csv')
list_erratic = pd.read_csv(f'{INPUT_DIR_2}/Erratic_ID_2_Year_Data.csv')
list_smooth = pd.read_csv(f'{INPUT_DIR_2}/Smooth_ID_2_Year_Data.csv')

list_intermittent = list_intermittent['0'].values.tolist()
list_lumpy = list_lumpy['0'].values.tolist()
list_erratic = list_erratic['0'].values.tolist()
list_smooth = list_smooth['0'].values.tolist()

sales_intermittent = sales_train_val[sales_train_val.id.isin(list_intermittent)]
sales_lumpy = sales_train_val[sales_train_val.id.isin(list_lumpy)]
sales_erratic = sales_train_val[sales_train_val.id.isin(list_erratic)]
sales_smooth = sales_train_val[sales_train_val.id.isin(list_smooth)]

# User-defined functions to calculate Metrics and Croston_TSB algorithm

In [6]:
ROUNDING_DECIMAL = 4

def mase_calculation(ts, prediction):
    divisor = 0
    for i in range(1, ts.shape[0]):
        divisor = divisor + abs(ts.iloc[i] - ts.iloc[i-1])
    divisor = divisor/(ts.shape[0] - 1)
    diff    = abs(ts - prediction[:ts.shape[0]])/divisor
    mase    = diff.mean()
    return mase

def mape_calculation(actual, pred): 
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    mask = (actual != 0)
    return round((np.fabs(actual - pred)/actual)[mask].mean()*100, ROUNDING_DECIMAL)

def wmape_calculation(actual, pred):
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    return round((np.sum(np.absolute(actual-pred))/np.sum(actual))*100, ROUNDING_DECIMAL)

def smape_calculation(actual, predicted):
    if not all([isinstance(actual, np.ndarray), isinstance(predicted, np.ndarray)]):
        actual, predicted = np.array(actual), np.array(predicted)
    return round(np.mean(np.abs(predicted - actual) / ((np.abs(predicted) + np.abs(actual))/2))*100, ROUNDING_DECIMAL)

In [7]:
def Croston_TSB(ts,extra_periods=1,alpha=0.4,beta=0.4):
    d = np.array(ts) # Transform the input into a numpy array
    cols = len(d) # Historical period length
    d = np.append(d,[np.nan]*extra_periods) # Append np.nan into the demand array to cover future periods
    
    #level (a), probability(p) and forecast (f)
    a,p,f = np.full((3,cols+extra_periods),np.nan)
# Initialization
    first_occurence = np.argmax(d[:cols]>0)
    a[0] = d[first_occurence]
    p[0] = 1/(1 + first_occurence)
    f[0] = p[0]*a[0]
                 
    # Create all the t+1 forecasts
    for t in range(0,cols): 
        if d[t] > 0:
            a[t+1] = alpha*d[t] + (1-alpha)*a[t] 
            p[t+1] = beta*(1) + (1-beta)*p[t]  
        else:
            a[t+1] = a[t]
            p[t+1] = (1-beta)*p[t]       
        f[t+1] = p[t+1]*a[t+1]
        
    # Future Forecast
    a[cols+1:cols+extra_periods] = a[cols]
    p[cols+1:cols+extra_periods] = p[cols]
    f[cols+1:cols+extra_periods] = f[cols]
                      
    df = pd.DataFrame.from_dict({"Demand":d,"Forecast":f,"Period":p,"Level":a,"Error":d-f})
    return df

# Set value for parameters

In [8]:
list_params_alpha = [round(item, 1) for item in list(np.arange(0.1, 1, 0.1))]
list_params_beta = [round(item, 1) for item in list(np.arange(0.1, 1, 0.1))]

In [9]:
start_train_date = '2014-04-11'
end_train_date = '2016-04-10'
n_pred_days = 14
num_train_needed = 100
validation_training_ratio = 0.95

# Pipeline for Croston_TSB

In [10]:
def CrostonTSB_output_all_params(sales_pattern, list_params_alpha, list_params_beta, 
                                 start_train_date, end_train_date, n_pred_days, 
                                 num_train_needed, validation_training_ratio):
    
    sales_pattern_py = sales_pattern.copy()
    list_pattern_py = sales_pattern_py.id.unique().tolist()
    
    sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
    df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
    df_pattern.drop('Date', axis=1, inplace=True)
    df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
    df_pattern.index.names = ['Date']
    df_pattern = df_pattern.astype('float64')
    
    train_data = df_pattern[(df_pattern.index >= start_train_date) & (df_pattern.index <= end_train_date)]
    test_data = df_pattern.iloc[n_pred_days*-1:]
    train_data_first_part = train_data.iloc[:int(len(train_data)*validation_training_ratio), :]
    train_data_second_part = train_data.iloc[int(len(train_data)*validation_training_ratio):, :]
    
    df_result_all_params = pd.DataFrame()
    for product in list_pattern_py:
        train_first = train_data_first_part[product]
        train_second = train_data_second_part[product]
        for i in list_params_alpha:
            for j in list_params_beta:
                predictions = list()
                history = [x for x in train_first[num_train_needed*-1:]]
                for t in range(len(train_second)):
                    yhat = Croston_TSB(history, extra_periods=1, alpha = i, beta = j)['Forecast'].iloc[-1]
                    predictions.append(yhat)
                    history.append(train_second[t])
                df_result_temp = pd.DataFrame({'Product': [product for count in range(len(train_second))],
                                               'Actual Data': train_second,
                                               'Forecast': predictions,
                                               'Alpha': [i for count_i in range(len(train_second))],
                                               'Beta': [j for count_j in range(len(train_second))]})
                df_result_all_params = df_result_all_params.append(df_result_temp, ignore_index=True)
            
    return df_result_all_params

In [11]:
start = timer()

pattern_df_result_all_params = CrostonTSB_output_all_params(sales_smooth,
                                                          list_params_alpha,
                                                          list_params_beta,
                                                          start_train_date,
                                                          end_train_date, 
                                                          n_pred_days,
                                                          num_train_needed,
                                                          validation_training_ratio)

end = timer()

print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 21.449098518799996 minutes


In [12]:
pattern_df_result_all_params

,Product,Actual Data,Forecast,Alpha,Beta
0,HOUSEHOLD_1_007_CA_1_validation,6.0,2.280631e+00,0.1,0.1
1,HOUSEHOLD_1_007_CA_1_validation,2.0,2.574464e+00,0.1,0.1
2,HOUSEHOLD_1_007_CA_1_validation,0.0,2.565101e+00,0.1,0.1
3,HOUSEHOLD_1_007_CA_1_validation,2.0,2.308591e+00,0.1,0.1
4,HOUSEHOLD_1_007_CA_1_validation,3.0,2.326776e+00,0.1,0.1
...,...,...,...,...,...
1156837,HOUSEHOLD_2_176_CA_4_validation,2.0,9.001153e-09,0.9,0.9
1156838,HOUSEHOLD_2_176_CA_4_validation,3.0,1.710012e+00,0.9,0.9
1156839,HOUSEHOLD_2_176_CA_4_validation,3.0,2.861101e+00,0.9,0.9
1156840,HOUSEHOLD_2_176_CA_4_validation,2.0,2.986011e+00,0.9,0.9


In [13]:
pattern_df_result_all_params.to_csv('CrostonTSB_Smooth_All_Parameters_Hyperparameter_Tuning_Data.csv')

# Summary all metrics

In [14]:
def get_metrics_result_all_params(data):
    metrics_df = {}
    metrics_df['MASE'] = mase_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['WMAPE'] = wmape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['SMAPE'] = smape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAPE'] = mape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAE'] = mean_absolute_error(data['Actual Data'], data['Forecast'])
    metrics_df['RMSE'] = np.sqrt(mean_squared_error(data['Actual Data'], data['Forecast']))
    return pd.Series(metrics_df)

In [15]:
start = timer()

df_result_metrics_all_params = pattern_df_result_all_params.groupby(['Product', 'Alpha', 'Beta']).apply(get_metrics_result_all_params).reset_index()

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 1.2978600489833336 minutes


In [16]:
df_result_metrics_all_params

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,HOUSEHOLD_1_003_CA_3_validation,0.1,0.1,0.716845,88.1220,108.4597,37.6178,1.214655,1.449333
1,HOUSEHOLD_1_003_CA_3_validation,0.1,0.2,0.726773,89.3424,109.1642,37.8431,1.231477,1.500788
2,HOUSEHOLD_1_003_CA_3_validation,0.1,0.3,0.744337,91.5015,111.0027,39.3262,1.261237,1.551801
3,HOUSEHOLD_1_003_CA_3_validation,0.1,0.4,0.764988,94.0402,113.2694,41.3380,1.296229,1.601942
4,HOUSEHOLD_1_003_CA_3_validation,0.1,0.5,0.786999,96.7460,115.8239,43.6005,1.333526,1.651234
...,...,...,...,...,...,...,...,...,...
31261,HOUSEHOLD_2_513_CA_3_validation,0.9,0.5,1.015293,95.3539,158.1019,106.4100,1.494737,2.653884
31262,HOUSEHOLD_2_513_CA_3_validation,0.9,0.6,1.006598,94.5373,157.6987,105.6663,1.481936,2.659324
31263,HOUSEHOLD_2_513_CA_3_validation,0.9,0.7,0.993743,93.3300,157.0807,103.9027,1.463010,2.665451
31264,HOUSEHOLD_2_513_CA_3_validation,0.9,0.8,0.979355,91.9787,156.0931,101.6825,1.441828,2.678167


# Check what products has unexpected metrics result

In [17]:
df_result_metrics_all_params[df_result_metrics_all_params['MASE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
324,HOUSEHOLD_1_018_CA_3_validation,0.1,0.1,NaN,NaN,NaN,NaN,0.0,0.0
325,HOUSEHOLD_1_018_CA_3_validation,0.1,0.2,NaN,NaN,NaN,NaN,0.0,0.0
326,HOUSEHOLD_1_018_CA_3_validation,0.1,0.3,NaN,NaN,NaN,NaN,0.0,0.0
327,HOUSEHOLD_1_018_CA_3_validation,0.1,0.4,NaN,NaN,NaN,NaN,0.0,0.0
328,HOUSEHOLD_1_018_CA_3_validation,0.1,0.5,NaN,NaN,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...
31099,HOUSEHOLD_2_490_CA_3_validation,0.9,0.5,NaN,NaN,NaN,NaN,0.0,0.0
31100,HOUSEHOLD_2_490_CA_3_validation,0.9,0.6,NaN,NaN,NaN,NaN,0.0,0.0
31101,HOUSEHOLD_2_490_CA_3_validation,0.9,0.7,NaN,NaN,NaN,NaN,0.0,0.0
31102,HOUSEHOLD_2_490_CA_3_validation,0.9,0.8,NaN,NaN,NaN,NaN,0.0,0.0


In [18]:
df_result_metrics_all_params[df_result_metrics_all_params['WMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
324,HOUSEHOLD_1_018_CA_3_validation,0.1,0.1,NaN,NaN,NaN,NaN,0.0,0.0
325,HOUSEHOLD_1_018_CA_3_validation,0.1,0.2,NaN,NaN,NaN,NaN,0.0,0.0
326,HOUSEHOLD_1_018_CA_3_validation,0.1,0.3,NaN,NaN,NaN,NaN,0.0,0.0
327,HOUSEHOLD_1_018_CA_3_validation,0.1,0.4,NaN,NaN,NaN,NaN,0.0,0.0
328,HOUSEHOLD_1_018_CA_3_validation,0.1,0.5,NaN,NaN,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...
31099,HOUSEHOLD_2_490_CA_3_validation,0.9,0.5,NaN,NaN,NaN,NaN,0.0,0.0
31100,HOUSEHOLD_2_490_CA_3_validation,0.9,0.6,NaN,NaN,NaN,NaN,0.0,0.0
31101,HOUSEHOLD_2_490_CA_3_validation,0.9,0.7,NaN,NaN,NaN,NaN,0.0,0.0
31102,HOUSEHOLD_2_490_CA_3_validation,0.9,0.8,NaN,NaN,NaN,NaN,0.0,0.0


In [19]:
df_result_metrics_all_params[df_result_metrics_all_params['SMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
324,HOUSEHOLD_1_018_CA_3_validation,0.1,0.1,NaN,NaN,NaN,NaN,0.0,0.0
325,HOUSEHOLD_1_018_CA_3_validation,0.1,0.2,NaN,NaN,NaN,NaN,0.0,0.0
326,HOUSEHOLD_1_018_CA_3_validation,0.1,0.3,NaN,NaN,NaN,NaN,0.0,0.0
327,HOUSEHOLD_1_018_CA_3_validation,0.1,0.4,NaN,NaN,NaN,NaN,0.0,0.0
328,HOUSEHOLD_1_018_CA_3_validation,0.1,0.5,NaN,NaN,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...
31099,HOUSEHOLD_2_490_CA_3_validation,0.9,0.5,NaN,NaN,NaN,NaN,0.0,0.0
31100,HOUSEHOLD_2_490_CA_3_validation,0.9,0.6,NaN,NaN,NaN,NaN,0.0,0.0
31101,HOUSEHOLD_2_490_CA_3_validation,0.9,0.7,NaN,NaN,NaN,NaN,0.0,0.0
31102,HOUSEHOLD_2_490_CA_3_validation,0.9,0.8,NaN,NaN,NaN,NaN,0.0,0.0


In [20]:
df_result_metrics_all_params[df_result_metrics_all_params['MAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
324,HOUSEHOLD_1_018_CA_3_validation,0.1,0.1,NaN,NaN,NaN,NaN,0.0,0.0
325,HOUSEHOLD_1_018_CA_3_validation,0.1,0.2,NaN,NaN,NaN,NaN,0.0,0.0
326,HOUSEHOLD_1_018_CA_3_validation,0.1,0.3,NaN,NaN,NaN,NaN,0.0,0.0
327,HOUSEHOLD_1_018_CA_3_validation,0.1,0.4,NaN,NaN,NaN,NaN,0.0,0.0
328,HOUSEHOLD_1_018_CA_3_validation,0.1,0.5,NaN,NaN,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...
31099,HOUSEHOLD_2_490_CA_3_validation,0.9,0.5,NaN,NaN,NaN,NaN,0.0,0.0
31100,HOUSEHOLD_2_490_CA_3_validation,0.9,0.6,NaN,NaN,NaN,NaN,0.0,0.0
31101,HOUSEHOLD_2_490_CA_3_validation,0.9,0.7,NaN,NaN,NaN,NaN,0.0,0.0
31102,HOUSEHOLD_2_490_CA_3_validation,0.9,0.8,NaN,NaN,NaN,NaN,0.0,0.0


In [21]:
df_result_metrics_all_params[df_result_metrics_all_params['RMSE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


In [22]:
df_result_metrics_all_params[df_result_metrics_all_params['MAE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


# Metrics Statistics

In [23]:
list_metrics = ['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE']

In [24]:
for mts in list_metrics:
    print('Percentage of unexpected values of', mts, 'is: {}'.format(df_result_metrics_all_params[df_result_metrics_all_params[mts].isin([np.nan, np.inf, -np.inf])].Product.nunique() / df_result_metrics_all_params.Product.nunique() * 100), "%")

Percentage of unexpected values of MASE is: 19.94818652849741 %
Percentage of unexpected values of WMAPE is: 19.94818652849741 %
Percentage of unexpected values of SMAPE is: 18.911917098445596 %
Percentage of unexpected values of MAPE is: 19.94818652849741 %
Percentage of unexpected values of MAE is: 0.0 %
Percentage of unexpected values of RMSE is: 0.0 %


### Filter all rows that have unexpected metrics values

In [25]:
df_result_metrics_all_params = df_result_metrics_all_params[~df_result_metrics_all_params.isin([np.nan, np.inf, -np.inf]).any(1)]

### Get MEAN metrics value of each Alpha

In [26]:
df_result_metrics_all_params.groupby(['Alpha', 'Beta'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()

MASE      WMAPE      SMAPE       MAPE       MAE      RMSE
Alpha Beta                                                               
0.1   0.1   0.856736  74.635030  82.172759  62.091767  2.740696  3.439181
      0.2   0.846226  72.941417  82.825802  63.550079  2.739644  3.456769
      0.3   0.847419  73.169459  83.527971  64.867376  2.745784  3.479963
      0.4   0.850659  73.557229  84.351083  66.173845  2.757992  3.506754
      0.5   0.855239  73.996899  85.354211  67.535785  2.775165  3.536645
...              ...        ...        ...        ...       ...       ...
0.9   0.5   0.936579  78.502168  92.797081  77.095994  3.062540  3.949307
      0.6   0.942853  79.027919  93.899256  78.485343  3.088526  3.990954
      0.7   0.950473  79.670050  95.323251  79.986195  3.117866  4.036033
      0.8   0.959230  80.415155  97.171239  81.640495  3.149142  4.085624
      0.9   0.969651  81.307264  99.712140  83.590393  3.183296  4.141481

[81 rows x 6 columns]

In [27]:
for mts in list_metrics:
    print('The optimum Alpha and Beta based on Mean', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Alpha', 'Beta'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()[mts].idxmin()))

The optimum Alpha and Beta based on Mean MASE is: (0.1, 0.2)
The optimum Alpha and Beta based on Mean WMAPE is: (0.1, 0.2)
The optimum Alpha and Beta based on Mean SMAPE is: (0.1, 0.1)
The optimum Alpha and Beta based on Mean MAPE is: (0.1, 0.1)
The optimum Alpha and Beta based on Mean MAE is: (0.1, 0.2)
The optimum Alpha and Beta based on Mean RMSE is: (0.1, 0.1)


### Get MEDIAN metrics value of each Alpha

In [28]:
df_result_metrics_all_params.groupby(['Alpha', 'Beta'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()

MASE    WMAPE    SMAPE     MAPE       MAE      RMSE
Alpha Beta                                                         
0.1   0.1   0.796849  62.7037  73.6864  58.7577  1.872334  2.373243
      0.2   0.801631  62.7254  73.9514  59.7578  1.917102  2.381027
      0.3   0.803525  62.9859  74.6079  60.6410  1.897476  2.407822
      0.4   0.805164  63.3248  75.2157  61.2834  1.899602  2.414768
      0.5   0.812991  63.8363  76.7723  62.4837  1.908638  2.421485
...              ...      ...      ...      ...       ...       ...
0.9   0.5   0.933082  72.6230  85.8961  72.9656  2.244907  2.817372
      0.6   0.939176  73.3686  86.9666  74.5968  2.269351  2.866077
      0.7   0.947261  74.1434  88.6753  76.2127  2.292462  2.911171
      0.8   0.955843  74.5104  90.7741  77.9255  2.314256  2.942473
      0.9   0.963399  75.3809  93.8565  79.6662  2.340108  2.978393

[81 rows x 6 columns]

In [29]:
for mts in list_metrics:
    print('The optimum Alpha and Beta based on Median of', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Alpha', 'Beta'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()[mts].idxmin()))

The optimum Alpha and Beta based on Median of MASE is: (0.1, 0.1)
The optimum Alpha and Beta based on Median of WMAPE is: (0.1, 0.1)
The optimum Alpha and Beta based on Median of SMAPE is: (0.1, 0.1)
The optimum Alpha and Beta based on Median of MAPE is: (0.1, 0.1)
The optimum Alpha and Beta based on Median of MAE is: (0.1, 0.1)
The optimum Alpha and Beta based on Median of RMSE is: (0.1, 0.1)


# Run Croston model after decide best paramaters

In [32]:
best_alpha = 0.1
best_beta = 0.1

In [33]:
def CrostonTSB_test_data(best_alpha, best_beta, sales_pattern,
                         start_train_date, end_train_date, 
                         n_pred_days, num_train_needed, validation_training_ratio):
    
    sales_pattern_py = sales_pattern.copy()
    list_pattern_py = sales_pattern_py.id.unique().tolist()
    
    sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
    df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
    df_pattern.drop('Date', axis=1, inplace=True)
    df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
    df_pattern.index.names = ['Date']
    df_pattern = df_pattern.astype('float64')
    
    train_data = df_pattern[(df_pattern.index >= start_train_date) & (df_pattern.index <= end_train_date)]
    test_data = df_pattern.iloc[n_pred_days*-1:]
    
    df_result_best_params = pd.DataFrame()
    for product in list_pattern_py:
        train_product = train_data[product]
        test_product = test_data[product]
        predictions = list()
        history = [x for x in train_product[num_train_needed*-1:]]
        for t in range(len(test_product)):
            yhat = Croston_TSB(history, extra_periods=1, alpha=best_alpha, beta = best_beta)['Forecast'].iloc[-1]
            predictions.append(yhat)
            history.append(test_product[t])
        df_result_temp = pd.DataFrame({'Product': [product for count in range(len(test_product))],
                                        'Actual Data': test_product,
                                        'Forecast': predictions})
        df_result_best_params = df_result_best_params.append(df_result_temp, ignore_index=True)
            
    return df_result_best_params

In [34]:
start = timer()

pattern_df_result_best_params = CrostonTSB_test_data(best_alpha, best_beta, sales_smooth, start_train_date, end_train_date, n_pred_days, num_train_needed, validation_training_ratio)

end = timer()

print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.08144019350000538 minutes


In [35]:
df_result_final = pattern_df_result_best_params.groupby('Product').apply(get_metrics_result_all_params).reset_index()

In [36]:
df_result_final

,Product,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,HOUSEHOLD_1_003_CA_3_validation,0.876545,151.0354,148.7927,44.3550,1.078824,1.126886
1,HOUSEHOLD_1_004_CA_3_validation,0.814881,63.6664,78.2435,54.4004,2.319275,2.952091
2,HOUSEHOLD_1_007_CA_1_validation,1.026687,49.7548,50.1835,50.8891,0.710783,0.892171
3,HOUSEHOLD_1_007_CA_3_validation,0.744155,89.8536,101.7579,66.0384,2.117978,2.427413
4,HOUSEHOLD_1_018_CA_3_validation,NaN,NaN,NaN,NaN,0.000000,0.000000
...,...,...,...,...,...,...,...
381,HOUSEHOLD_2_483_CA_3_validation,0.647436,58.1033,59.3587,61.3299,1.992112,2.704714
382,HOUSEHOLD_2_490_CA_2_validation,NaN,NaN,NaN,NaN,0.000000,0.000000
383,HOUSEHOLD_2_490_CA_3_validation,NaN,NaN,NaN,NaN,0.000000,0.000000
384,HOUSEHOLD_2_510_CA_3_validation,0.735011,46.5618,52.2578,47.0869,1.130786,1.360114


In [37]:
df_result_final.to_csv('CrostonTSB_Smooth_Test_Data.csv')